# Programmatic example with MARS/GRIB input

In [1]:
%%javascript
// leave this in to disable autoscroll in Jupyter notebook
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## Imports

In [2]:
import os
from pathlib import Path
import json
import numpy as np
import fcpy

/workspace/field-compression/miniconda3/envs/fcpy/lib/python3.10/site-packages/ecmwflibs/__init__.py:69: UserWarning: ecmwflibs: ignoring provided 'GRIB_DEFINITION_PATH' set to '/workspace/field-compression/miniconda3/envs/fcpy/lib/metview-bundle/share/metview/etc/grib_def/definitions:/workspace/field-compression/miniconda3/envs/fcpy/./share/eccodes/definitions:/tmp/mv.1945./bufr_def'. If you want ecmwflibs to use this environment variable, use ECMWFLIBS_GRIB_DEFINITION_PATH instead. If you want to use your own ECCODES library, use ECMWFLIBS_ECCODES.
  warnings.warn(


## Load data

In [3]:
# =================== TODO =================== #
# Note: can get access to the ECMWF API access #
# =================== TODO =================== #

if (Path().home() / ".ecmwfapirc").exists():
    fpath = "../samples/cases/hplp_ml_q.json"
    request = json.load(open(fpath))["mars"]
    ds = fcpy.fetch_mars_dataset(request)
else:
    ds = fcpy.open_dataset("../data/mars.grib")
    
ds = ds.isel(hybrid=[0, 1])
ds

In  , overriding the default value (cache=True) with cache=False is not recommended.


<xarray.Dataset>
Dimensions:     (time: 1, step: 1, hybrid: 2, values: 654400)
Coordinates:
  * time        (time) datetime64[ns] 2020-07-21
  * step        (step) timedelta64[ns] 00:00:00
  * hybrid      (hybrid) float64 1.0 2.0
    latitude    (values) float64 ...
    longitude   (values) float64 ...
    valid_time  (time, step) datetime64[ns] ...
Dimensions without coordinates: values
Data variables:
    q           (time, step, hybrid, values) float64 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-05-11T07:33 GRIB to CDM+CF via cfgrib-0.9.1...
    path:                    ../data/mars.grib

In [4]:
# Hybrid is a special type of levels
display(ds)

<xarray.Dataset>
Dimensions:     (time: 1, step: 1, hybrid: 2, values: 654400)
Coordinates:
  * time        (time) datetime64[ns] 2020-07-21
  * step        (step) timedelta64[ns] 00:00:00
  * hybrid      (hybrid) float64 1.0 2.0
    latitude    (values) float64 ...
    longitude   (values) float64 ...
    valid_time  (time, step) datetime64[ns] ...
Dimensions without coordinates: values
Data variables:
    q           (time, step, hybrid, values) float64 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-05-11T07:33 GRIB to CDM+CF via cfgrib-0.9.1...
    path:                    ../data/mars.grib

## Define and run experiment

In [5]:
suite = fcpy.Suite(
    ds,
    baseline=fcpy.Float(bits=32),
    # compressors can be composed: OuterCompressor(inner_compressor)
    compressors=[fcpy.LinQuantization(), fcpy.Round()],
    metrics=[fcpy.Difference, fcpy.AbsoluteError],
    bits=[13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
    # In this dataset, there are 451x900 points, f32 format
    max_chunk_size_bytes=451 * 900 * 6,
    # Optimisation
    skip_histograms=True,
)

  Activating new project at `/workspace/field-compression/miniconda3/envs/fcpy/lib/python3.10/site-packages`


ImportError: BitInformation not found

## Lineplot: evaluate the effect of bits

In [ ]:
# median/mean/std/var/q1/q3/min/max reduction
suite.lineplot(fcpy.AbsoluteError, reduction="max")

## Boxplot: evaluate statistical characteristics of errors

In [ ]:
bits = 13
suite.boxplot(metric=fcpy.Difference, compressor="LinQuantization", bits=bits)
suite.boxplot(metric=fcpy.Difference, compressor="Round", bits=bits)

## Spatial plots: evaluate the spatial distribution of errors

In [ ]:
if not os.getenv("SPHINX_BUILD"):
    # Here we compare the spatial change of specific humidity
    # by rounding values to 8 bits
    fcpy.spatialplot(
        ds=ds,
        baseline=fcpy.Float(bits=32),
        var_name="q",
        compressor=fcpy.LinQuantization(bits=13),
        metric=fcpy.Difference,
        latitude=0,
        longitude=0,
        hybrid=1,
        step=np.timedelta64(0, "ns"),
        # produces the bottom plots
        # bottom right plot shows error evolution across all levels, here just two levels
        third_dim="hybrid",
    )